In [188]:
# Dependencies
import pandas as pd
import re
import os

In [344]:
# Store filepath in a variable
file_path = "../Data/Toronto_FSA.csv"

In [345]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
file_df = pd.read_csv(file_path, encoding="ISO-8859-1")

In [346]:
# Show just the header
file_df.head()

,Prov/Terr,FSA,Total,Total Income,Average Income,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999",...,"$45,000 to $49,999","$50,000 to $54,999","$55,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 to $149,999","$150,000 to $249,999","$250,000 and over"
0,10,A0A,40150,1.673931e+09,41691.93026,2950,2700,3880,5220,3220,...,1600,1320,1210,1700,1380,1150,960,2240,880,170
1,10,A0B,17820,7.141520e+08,40075.86981,1180,1150,1790,2570,1650,...,710,580,480,710,520,420,340,960,380,60
2,10,A0C,11180,3.787120e+08,33874.06082,780,720,1220,1870,1180,...,440,300,270,390,230,200,180,340,110,10
3,10,A0E,19860,8.285010e+08,41717.06949,1520,1390,2070,2850,1850,...,610,460,450,710,610,580,530,1520,460,60
4,10,A0G,31290,1.089500e+09,34819.43113,2510,1960,3840,5040,2890,...,1110,850,670,1060,780,690,510,1230,340,70


In [347]:
file_df = file_df.reindex(sorted(file_df.columns), axis=1)

In [348]:
file_df.head()

,"$10,000 to $14,999","$100,000 to $149,999","$15,000 to $19,999","$150,000 to $249,999","$20,000 to $24,999","$25,000 to $29,999","$250,000 and over","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999",...,"$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999",Average Income,FSA,Prov/Terr,Total,Total Income,"Under $5,000"
0,3880,2240,5220,880,3220,2770,170,2550,2190,2090,...,1700,1380,1150,960,41691.93026,A0A,10,40150,1.673931e+09,2950
1,1790,960,2570,380,1650,1370,60,1130,960,860,...,710,520,420,340,40075.86981,A0B,10,17820,7.141520e+08,1180
2,1220,340,1870,110,1180,940,10,840,610,550,...,390,230,200,180,33874.06082,A0C,10,11180,3.787120e+08,780
3,2070,1520,2850,460,1850,1400,60,1140,880,800,...,710,610,580,530,41717.06949,A0E,10,19860,8.285010e+08,1520
4,3840,1230,5040,340,2890,2380,70,2180,1740,1450,...,1060,780,690,510,34819.43113,A0G,10,31290,1.089500e+09,2510


In [349]:
#select only FSA and Average Income
postal_codes_income = file_df[["FSA", "Average Income"]]

In [350]:
postal_codes_income = postal_codes_income.dropna(how='any',axis=0)

In [351]:
#Filter out the postal_codes_income based on FSAs what we have in the rantal and community tables
list_of_FSA = ["L0J","M1X","M1R","L4W","L9V","L7B","L4H","M3H","M5H","M3N","L6G","M5N","L5W","L5C","M9P","L1S","L5J","L1N",
"M4P","M5V","YO21","L5G","M8X","M4Y","M4H","M5J","M5M","L7A","M4W","L1Z","L7E","M3J","L5N","L3P","M4L","M6S","M2N","M1C",
"L5E","N7T","M6M","M6A","M3M","M6G","M4T","M4G","M1K","L6M","M3A","L4S","P2A","M6J","L6X","M5A","L9Z","L4X","L4E","M1V",
"M8Z","M4R","L0G","L1K","L1T","M5S","M8V","L6H","L5B","M4X","M9R","M4A","L0P","V6H","L6Y","L9S","L6B","L6L","L3T","L6E",
"L3X","L5R","L3R","L4G","M6P","K0M","M6N","M5T","L5K","M1B","M8W","M3C","P0S","M1N","M4V","L9C","L4Y","L0N","L3Y","M2R","L5A",
"M2M","L6S","M5B","L6Z","L6R","L6V","L1V","J3Y","M7A","M5E","M4B","P1P","M4E","M4M","M4J","L4B","M4S","L3S","M9M","M9C",
"L6T","L3Z","M9W","M6E","M3B","L4C","M1M","L5M","M5P","L6C","M2J","M6L","M9A","M2H","M1P","L0A","L6K","L1H","L1G","M5C","L4K",
"H4E","M2K","M2L","M1L","P6A","M4N","L4L","M1E","M1W","L6P","V9K","M9B","L6W","M6B","L6A","M6H","V9R","M1J","M4K","M3K",
"M1G","M6C","V0H","L0H","L7C","M5G","L9T","M5R","M1S","V4B","L4Z","M3L","L1W","M9V","L1J","M9N","L4J","M8Y","L5H","M6R","L5L"
"L4T","M1T","M4C","M6K","L1R","M1H","L1L","P0B","V8P","L5V", 'M2P', 'M5K', 'M5L', 'M5W', 'M5X', 'M7Y', 'M9L']

In [352]:
postal_codes_income = postal_codes_income[postal_codes_income['FSA'].isin(list_of_FSA)]

In [353]:
#Some of the data is in string format; Need to convert that to zero

In [354]:
postal_codes_income.loc[postal_codes_income[postal_codes_income['FSA'].isin(['M7A', 'M7Y'])].index,:]=[['M7A', 0], ['M7Y', 0]]

In [355]:
#Change the column names as per schema
postal_codes_income.columns = ["FSA", "avg_income"]

In [356]:
postal_codes_income.reset_index(drop=True, inplace=True)

In [359]:
postal_codes_income = postal_codes_income.append(pd.DataFrame([['L5LL4T',0], ['YO21', 0]], columns=postal_codes_income.columns), ignore_index=True)

In [369]:
#postal_codes_income.to_csv(os.path.join('..','Output','Toronto_FSA_Final.csv'), index=False)